# Tutorial 2 - A simple pipeline with signal postprocessing.

Here we demonstrate how to build a *MathAgent* as an intermediate to process the 
*SineGeneratorAgent*'s output before plotting.
Subsequently, a *MultiMathAgent* is built to show the ability to send a dictionary of 
multiple fields to the recipient. We provide a custom `on_received_message()` function, 
which is called every time a message is received from input agents.

The received message is a dictionary of the form:

    {
    'from':agent_name,
    'data': data,
    'senderType': agent_class_name,
    'channel':'channel_name'
    }

By default, `'channel'` is set to `"default"`, however a custom channel can be set 
when needed, which is demonstrated in the next tutorial.

In [1]:
# %load tutorial_2_math_agent.py
from agentMET4FOF.agents import (
    AgentMET4FOF,
    AgentNetwork,
    MonitorAgent,
    SineGeneratorAgent,
)


class MathAgent(AgentMET4FOF):
    def on_received_message(self, message):
        data = self.divide_by_two(message["data"])
        self.send_output(data)

    # Define simple math functions.
    @staticmethod
    def divide_by_two(numerator: float) -> float:
        return numerator / 2


class MultiMathAgent(AgentMET4FOF):

    _minus_param: float
    _plus_param: float

    def init_parameters(self, minus_param=0.5, plus_param=0.5):
        self._minus_param = minus_param
        self._plus_param = plus_param

    def on_received_message(self, message):
        minus_data = self.minus(message["data"], self._minus_param)
        plus_data = self.plus(message["data"], self._plus_param)

        self.send_output({"minus": minus_data, "plus": plus_data})

    @staticmethod
    def minus(minuend: float, subtrahend: float) -> float:
        return minuend - subtrahend

    @staticmethod
    def plus(summand_1: float, summand_2: float) -> float:
        return summand_1 + summand_2


def main():
    # start agent network server
    agentNetwork = AgentNetwork()
    # init agents
    gen_agent = agentNetwork.add_agent(agentType=SineGeneratorAgent)
    math_agent = agentNetwork.add_agent(agentType=MathAgent)
    multi_math_agent = agentNetwork.add_agent(agentType=MultiMathAgent)
    monitor_agent = agentNetwork.add_agent(agentType=MonitorAgent)
    # connect agents : We can connect multiple agents to any particular agent
    agentNetwork.bind_agents(gen_agent, math_agent)
    agentNetwork.bind_agents(gen_agent, multi_math_agent)
    # connect
    agentNetwork.bind_agents(gen_agent, monitor_agent)
    agentNetwork.bind_agents(math_agent, monitor_agent)
    agentNetwork.bind_agents(multi_math_agent, monitor_agent)
    # set all agents states to "Running"
    agentNetwork.set_running_state()

    # allow for shutting down the network after execution
    return agentNetwork


if __name__ == "__main__":
    main()

Starting NameServer...
Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:3333 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:3333

--------------------------------------------------------------
|                                                            |
| Your agent network is starting up. Open your browser and   |
| visit the agentMET4FOF dashboard on http://127.0.0.1:8050/ |
|                                                            |
--------------------------------------------------------------

INFO [2021-02-05 19:18:07.585019] (SineGeneratorAgent_1): INITIALIZED
INFO [2021-02-05 19:18:07.619684] (MathAgent_1): INITIALIZED
INFO [2021-02-05 19:18:07.654192] (MultiMathAgent_1): INITIALIZED
INFO [2021-02-05 19:18:07.691566] (MonitorAgent_1): INITIALIZED
[2021-02-05 19:18:07.706815] (SineGeneratorAgent_1): Connected output module: MathAgent_1
[2021-02-05 19:18:07.714664] (SineGeneratorAgent_1): Connected output module: MultiMathAgent_1
[2021-02-05 19:18:07.724529